<a href="https://colab.research.google.com/github/tidepool-org/data-analytics/blob/etn%2Fcolab-notebooks/projects/colab-notebooks/Get_your_Tidepool_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get your Tidepool Data


## Instructions

1.   Please run this in Playground Mode (click File, and Open in playground mode) or create a copy so you don't overwrite this example (i.e., File, "Save a copy in Drive..."
2.   Before you run the code be sure to select "Connect" in the top right-hand corner of the webpage.
3.   To run the code, you press shift-return to execute each cell, or you can press the Run button (above), or you can click on the 
4.   You will need to enter in your Tidepool credentials to pull your data (i.e., email address & pwd)
5.   You will be prompted to connect your collaboratory instance to google drive, follow the link and copy/paste the auth string back into this notebook.
6.   The code will save the results to your root google drive folder, "My Drive." 

## Note
* In each of the cells, functions are defined first folllwed by code that uses those functions







In [0]:
# %% REQUIRED LIBRARIES
import numpy as np
import pandas as pd
import os
import requests
import json
import getpass

# get data from Tidepool API

*   You can either enter in the big data credentials (Tidepool employees only) AND userID, or
*   You can enter in your Tidepool email (username) and password (when prompted)


In [0]:
# %% get Tidepool data from api
def get_data_from_api(
    email=np.nan,
    password=np.nan,
    years_of_data=10,
    userid_of_shared_user=np.nan,
):
  
    if pd.isnull(email):
        email=input("Enter Tidepool email address:\n")
      
    if pd.isnull(password):
        password=getpass.getpass("Enter password:\n")
  
    df = pd.DataFrame()
    url1 = "https://api.tidepool.org/auth/login"
    url3 = "https://api.tidepool.org/auth/logout"
    
    myResponse = requests.post(url1, auth=(email, password))

    if(myResponse.ok):
        xtoken = myResponse.headers["x-tidepool-session-token"]
        
        if pd.isnull(userid_of_shared_user):
            userid = json.loads(myResponse.content.decode())["userid"]
        else:
            userid = userid_of_shared_user       
        
        endDate = pd.datetime.now()

        for years in range(1, years_of_data + 1):
            startDate = endDate - pd.Timedelta(365, unit="d")

            url2 = "https://api.tidepool.org/data/" + userid + \
                "?endDate=" + endDate.strftime("%Y-%m-%d") + \
                "T23:59:59.000Z&startDate=" + \
                startDate.strftime("%Y-%m-%d") + "T00:00:00.000Z"

            headers = {
                "x-tidepool-session-token": xtoken,
                "Content-Type": "application/json"
                }

            myResponse2 = requests.get(url2, headers=headers)
            if(myResponse2.ok):

                usersData = json.loads(myResponse2.content.decode())
                tempDF = pd.DataFrame(usersData)
                df = pd.concat([df, tempDF], ignore_index=True)

            else:
                print("ERROR in year ", years, myResponse2.status_code)

            endDate = startDate - pd.Timedelta(1, unit="d")
    else:
        print("ERROR in getting token ", myResponse.status_code)
        myResponse2 = np.nan

    myResponse3 = requests.post(url3, auth=(email, password))
    
    responses = [myResponse, myResponse2, myResponse3]

    return df, responses
  
  
data, responses = get_data_from_api(
    years_of_data=1,  # valid numbers are 1-10
)
# print a list of the column headings in the dataset
list(data)

# save your data


In [0]:
# save the data
from google.colab import drive
drive.mount("drive", force_remount=True)

data.to_csv("your-tidepool-data.csv")
!cp your-tidepool-data.csv drive/My\ Drive/
